In [ ]:
!nvidia-smi

Wed May 31 03:21:01 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8    12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install  dgl -q -f https://data.dgl.ai/wheels/cu118/repo.html
!pip install  dglgo -q -f https://data.dgl.ai/wheels-test/repo.html

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.4/86.4 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.5/63.5 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.4/52.4 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.9/111.9 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 37.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 100.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 49.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 572.7/572.7 kB 43.4 MB/s eta 0:

In [ ]:
!pip install -U -q wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.5/206.5 kB 19.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.4 MB/s eta 0:00:00


In [ ]:
import os
os.environ["DGLBACKEND"] = "pytorch"

import dgl
import torch
import torch.nn.functional as F
import scipy.sparse as sp

# GNN Training

本章節將會進入到 GNN 的訓練環節，我們會使用以下 4 種任務帶大家認識 GNN 的訓練過程，並且，本章節使用 pytorch 作為後端框架

1. Node Classification/Regression
2. Edge Classification/Regression
3. Link Prediction
4. Graph Classification

## 節點分類

在節點分類的任務中，我們輸入一張圖，並對圖上的每個節點進行分類，所以我們不需要 `GraphDataLoader` 批次讀取圖資料，相反地，我們只要有圖的信息結構即可

### Example Dataset

在節點分類的例子中，我們使用 `CoraGraphDataset`，這是一個非常著名的節點分類圖，該資料蒐集了 2708 個機器學習論文 (nodes)，每篇論文來自七個不同的機器學習主題 (神經網絡、概率方法、遺傳算法、監督學習、無監督學習、強化學習和規則學習)，節點代表論文之間的引用關係，更具體的說，有以下特徵

- 節點數量: 2708
- 邊的數量: 10556
- 節點特徵維度: 1433
- 類別數量: 7

### Training phase

在訓練階段，我們首先對每個節點劃分到訓練、驗證、測試三個資料集中，在前向傳播階段 `forward(graph, inputs)`，我們需要輸入圖的結構信息 (方便做 message passing) 和節點特徵 (知道當前要預測哪個節點)

### Testing phase

同 training phase，只需輸入節點特徵與圖即可

In [ ]:
# utils.py
import random
import numpy as np
import torch
import dgl
import wandb

def seed_everything(seed):
    # Set Python random seed
    random.seed(seed)

    # Set NumPy random seed
    np.random.seed(seed)

    # Set PyTorch random seed for CPU and GPU
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

    # Set PyTorch deterministic operations for cudnn backend
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

    # Set DGL random seed
    # dgl.seed(seed)
    dgl.random.seed(seed)

def wandb_settings(key, config, project, entity, name):

    # Wandb Login
    wandb.login(key=key)

    # Initialize W&B
    run = wandb.init(
        config=config,
        project=project,
        entity=entity,
        name=name,
    )

def get_id(mask):
  return torch.LongTensor(np.nonzero(mask)).squeeze()
# seed_everything(42)

In [ ]:
# model.py
import dgl.nn as dglnn
import torch.nn as nn
import torch.nn.functional as F
class SAGE(nn.Module):
    def __init__(self, in_feats, hid_feats, out_feats):
        super().__init__()
        # 实例化SAGEConve，in_feats是输入特征的维度，out_feats是输出特征的维度，aggregator_type是聚合函数的类型
        self.conv1 = dglnn.SAGEConv(
            in_feats=in_feats, out_feats=hid_feats, aggregator_type='mean')
        self.conv2 = dglnn.SAGEConv(
            in_feats=hid_feats, out_feats=out_feats, aggregator_type='mean')

    def forward(self, graph, inputs):
        # 输入是节点的特征
        h = self.conv1(graph, inputs)
        h = F.relu(h)
        h = self.conv2(graph, h)
        return h

In [ ]:
# main.py
import argparse
import sys

sys.argv = ['main.py']

def parse_args():
    parser = argparse.ArgumentParser(description='DGL GNN node classification')
    parser.add_argument('--epochs', type=int, default=200, help='number of training epochs')
    parser.add_argument("--print_freq", type=int, default=10, help="print frequency")
    parser.add_argument('--save_name', type=str, default='model.pth', help='name of the file to save the trained model')
    parser.add_argument('--lr', type=float, default=1e-2, help='learning rate for the optimizer')
    parser.add_argument('--weight_decay', type=float, default=0, help='weight decay for the optimizer')
    parser.add_argument('--resume', type=bool, default=False, help='resume or not')
    parser.add_argument('--resume_path', type=str, default="best_model.pth", help='resume path')
    parser.add_argument('--num_classes', type=int, default=7, help='number of classes')
    parser.add_argument('--hidden_feats', type=int, default=100, help='number of hidden units')
    parser.add_argument('--model_name', type=str, default="tv_densenet121", help='model name')
    parser.add_argument('--device', type=str, default="cuda", help='device')
    parser.add_argument('--rank', type=int, default=0, help='the rank of the device, 0 if single GPU')
    parser.add_argument('--name', type=str, default="test", help='name for the wandb run')
    args = parser.parse_args()
    return args

def main():

    seed_everything(42)

    args = parse_args()

    d = vars(args)
    key = "Your API key"
    wandb_settings(key, d, "DGL-Cora-Node-Classification", "DDCVLAB", args.name)

    dataset = CoraGraphDataset()
    graph = dataset[0]

    features = graph.ndata["feat"]
    labels = graph.ndata["label"]

    train_mask = graph.ndata["train_mask"]
    val_mask = graph.ndata["val_mask"]
    test_mask = graph.ndata["test_mask"]

    model = SAGE(in_feats=features.shape[1], hid_feats=args.hidden_feats, out_feats=args.num_classes)
    if args.resume:
      state_dict = torch.load(args.resume_path)
      model.load_state_dict(state_dict, strict=True)
    if args.device == "cuda":
      device = f"cuda:{args.rank}"
      model.to(device)
      graph = graph.to(device)
      features = features.to(device)
      labels = labels.to(device)
      train_mask = train_mask.to(device)
      val_mask = val_mask.to(device)
      test_mask = test_mask.to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=args.lr, weight_decay=args.weight_decay)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.1, patience=5)

    best_val_acc = 0
    best_test_acc = 0

    for epoch in range(args.epochs):
        # Forward
        logits = model(graph, features)

        # Compute prediction
        pred = logits.argmax(1)

        # Compute loss
        # Note that you should only compute the losses of the nodes in the training set.
        train_loss = F.cross_entropy(logits[train_mask], labels[train_mask])

        # Compute accuracy on training/validation/test
        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()

        # Backward
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()

        # Validation
        with torch.no_grad():
          valid_loss = F.cross_entropy(logits[val_mask], labels[val_mask])
          val_acc = (pred[val_mask] == labels[val_mask]).float().mean()

        # Save the best validation accuracy and the corresponding test accuracy.
        if best_val_acc < val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), args.save_name)


        if epoch % args.print_freq == 0:
            print(
                "Epoch [{}] | loss: {:.3f}, val acc: {:.3f} (best {:.3f})".format(
                    epoch, valid_loss, val_acc, best_val_acc
                )
            )

        wandb.log({"Train ACC": train_acc, "Epoch": epoch})
        wandb.log({"Val ACC": val_acc, "Epoch": epoch})

        wandb.log({"Train Loss": train_loss, "Epoch": epoch})
        wandb.log({"Val Loss": valid_loss, "Epoch": epoch})

        scheduler.step(valid_loss)

    # Testing
    state_dict = torch.load(args.save_name)
    model.load_state_dict(state_dict)
    model.eval()

    logits = model(graph, features)
    pred = logits.argmax(1)
    test_acc = (pred[test_mask] == labels[test_mask]).float().mean()

    print(f"Test ACC: {test_acc}")
    wandb.log({"Test ACC": test_acc})

if __name__ == "__main__":
    main()

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ddcvlab. Use `wandb login --relogin` to force relogin


NameError: ignored

### Node Classification on Heterograph

接下來我們看看怎麼在異構圖上面做節點分類，我們首先定義異構圖

In [ ]:
import numpy as np
import torch
from sklearn.model_selection import train_test_split

def build_heterograph():
  n_users = 1000
  n_items = 500
  n_follows = 3000
  n_clicks = 5000
  n_dislikes = 500
  n_hetero_features = 10
  n_user_classes = 5
  n_max_clicks = 10

  follow_src = np.random.randint(0, n_users, n_follows)
  follow_dst = np.random.randint(0, n_users, n_follows)
  click_src = np.random.randint(0, n_users, n_clicks)
  click_dst = np.random.randint(0, n_items, n_clicks)
  dislike_src = np.random.randint(0, n_users, n_dislikes)
  dislike_dst = np.random.randint(0, n_items, n_dislikes)

  hetero_graph = dgl.heterograph({
      ('user', 'follow', 'user'): (follow_src, follow_dst),
      ('user', 'followed-by', 'user'): (follow_dst, follow_src),
      ('user', 'click', 'item'): (click_src, click_dst),
      ('item', 'clicked-by', 'user'): (click_dst, click_src),
      ('user', 'dislike', 'item'): (dislike_src, dislike_dst),
      ('item', 'disliked-by', 'user'): (dislike_dst, dislike_src)})

  hetero_graph.nodes['user'].data['feature'] = torch.randn(n_users, n_hetero_features)
  hetero_graph.nodes['item'].data['feature'] = torch.randn(n_items, n_hetero_features)
  hetero_graph.nodes['user'].data['label'] = torch.randint(0, n_user_classes, (n_users,))
  hetero_graph.edges['click'].data['label'] = torch.randint(1, n_max_clicks, (n_clicks,)).float()
  # 在user类型的节点和click类型的边上随机生成训练集的掩码
  train_mask, valid_mask, test_mask = gen_mask(n_users)
  hetero_graph.nodes['user'].data['train_mask'] = train_mask
  hetero_graph.nodes['user'].data['val_mask'] = valid_mask
  hetero_graph.nodes['user'].data['test_mask'] = test_mask
  train_mask, valid_mask, test_mask = gen_mask(n_clicks)
  hetero_graph.edges['click'].data['train_mask'] = train_mask
  hetero_graph.edges['click'].data['val_mask'] = valid_mask
  hetero_graph.edges['click'].data['test_mask'] = test_mask
  return hetero_graph

def gen_mask(n):
  mask = torch.zeros(n, dtype=torch.bool)
  perm = torch.randperm(n)
  train_indices = perm[:int(0.6 * n)]
  val_indices = perm[int(0.6 * n):int(0.8 * n)]
  test_indices = perm[int(0.8 * n):]
  train_mask = torch.clone(mask)
  valid_mask = torch.clone(mask)
  test_mask = torch.clone(mask)
  train_mask[train_indices] = True
  valid_mask[val_indices] = True
  test_mask[test_indices] = True
  return train_mask, valid_mask, test_mask


In [ ]:
# Define a Heterograph Conv model

class RGCN(nn.Module):
    def __init__(self, in_feats, hid_feats, out_feats, rel_names):
        super().__init__()
        # 实例化HeteroGraphConv，in_feats是输入特征的维度，out_feats是输出特征的维度，aggregate是聚合函数的类型
        self.conv1 = dglnn.HeteroGraphConv({
            rel: dglnn.GraphConv(in_feats, hid_feats)
            for rel in rel_names}, aggregate='sum')
        self.conv2 = dglnn.HeteroGraphConv({
            rel: dglnn.GraphConv(hid_feats, out_feats)
            for rel in rel_names}, aggregate='sum')

    def forward(self, graph, inputs):
        # 输入是节点的特征字典
        h = self.conv1(graph, inputs)
        h = {k: F.relu(v) for k, v in h.items()}
        h = self.conv2(graph, h)
        return h

對於異構圖的資料結構，我們使用字典的形式儲存，我們也會用一個字典儲存模型對於不同實體類別的輸出，並且在測試階段，我們可以只關注於特定實體上的分類成效

In [ ]:
# main.py
import argparse
import sys

sys.argv = ['main.py']

def parse_args():
    parser = argparse.ArgumentParser(description='DGL GNN Heterograph node classification')
    parser.add_argument('--epochs', type=int, default=200, help='number of training epochs')
    parser.add_argument("--print_freq", type=int, default=10, help="print frequency")
    parser.add_argument('--save_name', type=str, default='model.pth', help='name of the file to save the trained model')
    parser.add_argument('--lr', type=float, default=1e-2, help='learning rate for the optimizer')
    parser.add_argument('--weight_decay', type=float, default=0, help='weight decay for the optimizer')
    parser.add_argument('--resume', type=bool, default=False, help='resume or not')
    parser.add_argument('--resume_path', type=str, default="best_model.pth", help='resume path')
    parser.add_argument('--num_classes', type=int, default=7, help='number of classes')
    parser.add_argument('--hidden_feats', type=int, default=100, help='number of hidden units')
    parser.add_argument('--model_name', type=str, default="tv_densenet121", help='model name')
    parser.add_argument('--device', type=str, default="cuda", help='device')
    parser.add_argument('--rank', type=int, default=0, help='the rank of the device, 0 if single GPU')
    parser.add_argument('--name', type=str, default="test", help='name for the wandb run')
    args = parser.parse_args()
    return args

def main():

    seed_everything(42)

    args = parse_args()

    d = vars(args)
    key = "Your API Key"
    wandb_settings(key, d, "DGL-Cora-Heterograph-Node-Classification", "DDCVLAB", args.name)

    hetero_graph = build_heterograph()

    user_feats = hetero_graph.nodes['user'].data['feature']
    item_feats = hetero_graph.nodes['item'].data['feature']
    features = {'user': user_feats, 'item': item_feats}
    labels = hetero_graph.nodes['user'].data['label']

    train_mask = hetero_graph.nodes['user'].data['train_mask']
    val_mask = hetero_graph.nodes['user'].data['val_mask']
    test_mask = hetero_graph.nodes['user'].data['test_mask']

    model = RGCN(10, 20, 5, hetero_graph.etypes)
    if args.resume:
      state_dict = torch.load(args.resume_path)
      model.load_state_dict(state_dict, strict=True)

    if args.device=="cuda":
      device = f"cuda:{args.rank}"
    else:
      device = args.device
    model.to(device)
    hetero_graph = hetero_graph.to(device)
    user_feats = user_feats.to(device)
    item_feats = item_feats.to(device)
    labels = labels.to(device)
    features = {k: v.to(device=device, non_blocking=True) if hasattr(v, 'to') else v for k, v in features.items()}
    train_mask = train_mask.to(device)
    val_mask = val_mask.to(device)
    test_mask = test_mask.to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=args.lr, weight_decay=args.weight_decay)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.1, patience=5)

    best_val_acc = 0
    best_test_acc = 0

    for epoch in range(args.epochs):
        # Forward
        logits = model(hetero_graph, features)["user"]
        # Compute prediction
        pred = logits.argmax(1)

        # Compute loss
        # Note that you should only compute the losses of the nodes in the training set.
        train_loss = F.cross_entropy(logits[train_mask], labels[train_mask])

        # Compute accuracy on training/validation/test
        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()

        # Backward
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()

        # Validation
        with torch.no_grad():
          valid_loss = F.cross_entropy(logits[val_mask], labels[val_mask])
          val_acc = (pred[val_mask] == labels[val_mask]).float().mean()

        # Save the best validation accuracy and the corresponding test accuracy.
        if best_val_acc < val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), args.save_name)


        if epoch % args.print_freq == 0:
            print(
                "Epoch [{}] | loss: {:.3f}, val acc: {:.3f} (best {:.3f})".format(
                    epoch, valid_loss, val_acc, best_val_acc
                )
            )

        wandb.log({"Train ACC": train_acc, "Epoch": epoch})
        wandb.log({"Val ACC": val_acc, "Epoch": epoch})

        wandb.log({"Train Loss": train_loss, "Epoch": epoch})
        wandb.log({"Val Loss": valid_loss, "Epoch": epoch})

        scheduler.step(valid_loss)

    # Testing
    state_dict = torch.load(args.save_name)
    model.load_state_dict(state_dict)
    model.eval()

    logits = model(hetero_graph, features)["user"]
    pred = logits.argmax(1)
    test_acc = (pred[test_mask] == labels[test_mask]).float().mean()

    print(f"Test ACC: {test_acc}")
    wandb.log({"Test ACC": test_acc})

if __name__ == "__main__":
    main()

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ddcvlab. Use `wandb login --relogin` to force relogin


NameError: ignored

Error in callback <function _WandbInit._pause_backend at 0x7f6302995120> (for post_run_cell):


KeyboardInterrupt: ignored

## 邊分類

在邊分類的任務中，我們輸入一張圖，並對圖上的每個邊進行分類，所以我們不需要 `GraphDataLoader` 批次讀取圖資料，相反地，我們只要有圖的信息結構即可

### Example Dataset

在邊分類的例子中，我們使用 `WN18Dataset`，這是一個非常著名的邊分類圖，該資料蒐集了廣泛的英語詞彙 (nodes)，總共有 18 種類別，約 40000 種實體

### Training phase

在訓練階段，我們首先對每個節點劃分到訓練、驗證、測試三個資料集中，在前向傳播階段 `forward(graph, inputs)`，我們需要輸入圖的結構信息 (方便做 message passing) 和節點特徵 (知道當前要預測哪個節點)

### Testing phase

同 training phase，只需輸入節點特徵與圖即可

In [ ]:
from dgl.data import WN18Dataset

In [ ]:
class TwoLayerGCN(nn.Module):
    def __init__(self, in_dim, hid_dim, out_dim):
        """两层的GCN模型"""
        super().__init__()
        self.conv1 = dglnn.GraphConv(in_dim, hid_dim, allow_zero_in_degree=True)
        self.conv2 = dglnn.GraphConv(hid_dim, out_dim, allow_zero_in_degree=True)

    def forward(self, blocks, x):
        x = F.relu(self.conv1(blocks[0], x))
        x = F.relu(self.conv2(blocks[1], x))
        return x

class Predictor(nn.Module):
    """边预测模块，将边两端节点表示拼接，然后接一个线性变换，得到最后的分类表示输出"""
    def __init__(self, in_dim, num_classes):
        super().__init__()
        self.W = nn.Linear(2 * in_dim, num_classes)

    def apply_edges(self, edges):
        data = torch.cat([edges.src['x'], edges.dst['x']], dim=-1)
        return {'score': self.W(data)}

    def forward(self, edge_subgraph, x):
        with edge_subgraph.local_scope():
            edge_subgraph.ndata['x'] = x
            edge_subgraph.apply_edges(self.apply_edges)
            return edge_subgraph.edata['score']

class MyModel(nn.Module):
    """主模型：结构比较清晰"""
    def __init__(self, emb_dim, hid_dim, out_dim, num_classes, num_nodes):
        super().__init__()
        self.node_emb = nn.Embedding(num_nodes, emb_dim)
        self.gcn = TwoLayerGCN(emb_dim, hid_dim, out_dim)
        self.predictor = Predictor(out_dim, num_classes)

    def forward(self, edge_subgraph, blocks, input_nodes):
        x = self.node_emb(input_nodes)
        x = self.gcn(blocks, x)
        return self.predictor(edge_subgraph, x)

In [ ]:
# main.py
import argparse
import sys
from sklearn.metrics import accuracy_score
import warnings

warnings.filterwarnings("ignore")

sys.argv = ['main.py']

def parse_args():
    parser = argparse.ArgumentParser(description='DGL GNN Heterograph node classification')
    parser.add_argument('--epochs', type=int, default=50, help='number of training epochs')
    parser.add_argument('--batch_size', type=int, default=2048, help='batch size')
    parser.add_argument("--print_freq", type=int, default=1, help="print frequency")
    parser.add_argument('--save_name', type=str, default='model.pth', help='name of the file to save the trained model')
    parser.add_argument('--lr', type=float, default=1e-2, help='learning rate for the optimizer')
    parser.add_argument('--weight_decay', type=float, default=0, help='weight decay for the optimizer')
    parser.add_argument('--resume', type=bool, default=False, help='resume or not')
    parser.add_argument('--resume_path', type=str, default="best_model.pth", help='resume path')
    parser.add_argument('--num_classes', type=int, default=7, help='number of classes')
    parser.add_argument('--hidden_feats', type=int, default=64, help='number of hidden units')
    parser.add_argument('--model_name', type=str, default="tv_densenet121", help='model name')
    parser.add_argument('--device', type=str, default="cuda", help='device')
    parser.add_argument('--rank', type=int, default=0, help='the rank of the device, 0 if single GPU')
    parser.add_argument('--name', type=str, default="test", help='name for the wandb run')
    parser.add_argument('--fanout', type=list, default=[10,25], help='number of samples in training steps')
    args = parser.parse_args()
    return args

def main():

    seed_everything(42)

    args = parse_args()

    # d = vars(args)
    # key = "Your API key"
    # wandb_settings(key, d, "DGL-Cora-Heterograph-Node-Classification", "DDCVLAB", args.name)

    dataset = WN18Dataset()
    graph = dataset[0]

    train_mask = graph.edata['train_edge_mask']
    val_mask = graph.edata['valid_edge_mask']
    test_mask = graph.edata['test_edge_mask']

    sampler = dgl.dataloading.NeighborSampler(args.fanout)
    sampler = dgl.dataloading.as_edge_prediction_sampler(sampler)
    train_dataloader = dgl.dataloading.DataLoader(
        graph,
        get_id(train_mask),
        sampler,
        batch_size=args.batch_size,
        shuffle=True,
        drop_last=False,
    )
    sampler = dgl.dataloading.MultiLayerFullNeighborSampler(2)
    sampler = dgl.dataloading.as_edge_prediction_sampler(sampler)
    valid_dataloader = dgl.dataloading.DataLoader(
        graph,
        get_id(val_mask),
        sampler,
        batch_size = args.batch_size
    )
    sampler = dgl.dataloading.MultiLayerFullNeighborSampler(2)
    sampler = dgl.dataloading.as_edge_prediction_sampler(sampler)
    test_dataloader = dgl.dataloading.DataLoader(
        graph,
        get_id(test_mask),
        sampler,
        batch_size = args.batch_size
    )


    model = MyModel(64, args.hidden_feats, args.hidden_feats, 18, graph.num_nodes())
    if args.resume:
      state_dict = torch.load(args.resume_path)
      model.load_state_dict(state_dict, strict=True)

    device = args.device
    model.to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=args.lr, weight_decay=args.weight_decay)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.1, patience=5)

    best_val_acc = 0
    best_test_acc = 0
    trn_label, trn_pred = [], []
    train_epoch_loss = 0
    valid_epoch_loss = 0
    for epoch in range(args.epochs):
        model.train()
        for step, (input_nodes, edges_subgraph, blocks) in enumerate(train_dataloader):
            # Forward
            edges_subgraph = edges_subgraph.to(device)
            blocks = [block.to(device) for block in blocks]
            input_nodes = input_nodes.to(device)

            logits = model(edges_subgraph, blocks, input_nodes)
            # Compute prediction
            pred = logits.argmax(1)

            # Compute loss
            # Note that you should only compute the losses of the nodes in the training set.
            train_loss = F.cross_entropy(logits, edges_subgraph.edata["etype"])
            train_epoch_loss += train_loss.item()

            # Compute accuracy on training/validation/test
            trn_label.extend(edges_subgraph.edata['etype'].cpu().numpy().tolist())
            trn_pred.extend(pred.detach().cpu().numpy().tolist())

            # Backward
            optimizer.zero_grad()
            train_loss.backward()
            optimizer.step()

        train_acc = accuracy_score(trn_label, trn_pred)

        model.eval()
        with torch.no_grad():
            for step, (input_nodes, edges_subgraph, blocks) in enumerate(valid_dataloader):
                input_nodes = input_nodes.to(device)
                edges_subgraph = edges_subgraph.to(device)
                blocks = [block.to(device) for block in blocks]

                # Validation
                logits = model(edges_subgraph, blocks, input_nodes)
                pred = logits.argmax(1)
                valid_loss = F.cross_entropy(logits, edges_subgraph.edata["etype"])
                valid_epoch_loss += valid_loss.item()
                trn_label.extend(edges_subgraph.edata['etype'].cpu().numpy().tolist())
                trn_pred.extend(pred.detach().cpu().numpy().tolist())

        val_acc = accuracy_score(trn_label, trn_pred)

        # Save the best validation accuracy and the corresponding test accuracy.
        if best_val_acc < val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), args.save_name)
        if epoch % args.print_freq == 0:
            print(
                "Epoch [{}] | loss: {:.3f}, val acc: {:.3f} (best {:.3f})".format(
                    epoch, valid_loss, val_acc, best_val_acc
                )
            )
        # wandb.log({"Train ACC": train_acc, "Epoch": epoch})
        # wandb.log({"Val ACC": val_acc, "Epoch": epoch})
        # wandb.log({"Train Loss": train_loss, "Epoch": epoch})
        # wandb.log({"Val Loss": valid_loss, "Epoch": epoch})

        scheduler.step(valid_loss)

    # Testing
    state_dict = torch.load(args.save_name)
    model.load_state_dict(state_dict)
    model.eval()

    trn_label, trn_pred = [], []
    with torch.no_grad():
        for step, (input_nodes, edges_subgraph, blocks) in enumerate(test_dataloader):
            input_nodes = input_nodes.to(device)
            edges_subgraph = edges_subgraph.to(device)
            blocks = [block.to(device) for block in blocks]
            logits = model(edges_subgraph, blocks, input_nodes)
            pred = logits.argmax(1)
            trn_label.extend(edges_subgraph.edata['etype'].cpu().numpy().tolist())
            trn_pred.extend(pred.detach().cpu().numpy().tolist())

    test_acc = accuracy_score(trn_label, trn_pred)

    print(f"Test ACC: {test_acc}")
    # wandb.log({"Test ACC": test_acc})

if __name__ == "__main__":
    main()

# entities: 40943
# relations: 18
# training edges: 141442
# validation edges: 5000
# testing edges: 5000
Done loading data from cached files.
Epoch [0] | loss: 1.312, val acc: 0.507 (best 0.507)
Epoch [1] | loss: 0.962, val acc: 0.598 (best 0.598)
Epoch [2] | loss: 0.989, val acc: 0.651 (best 0.651)
Epoch [3] | loss: 0.991, val acc: 0.685 (best 0.685)
Epoch [4] | loss: 0.928, val acc: 0.711 (best 0.711)
Epoch [5] | loss: 0.720, val acc: 0.731 (best 0.731)
Epoch [6] | loss: 0.781, val acc: 0.748 (best 0.748)
Epoch [7] | loss: 0.759, val acc: 0.762 (best 0.762)
Epoch [8] | loss: 0.582, val acc: 0.774 (best 0.774)
Epoch [9] | loss: 0.715, val acc: 0.784 (best 0.784)
Epoch [10] | loss: 0.697, val acc: 0.794 (best 0.794)
Epoch [11] | loss: 0.637, val acc: 0.802 (best 0.802)
Epoch [12] | loss: 0.622, val acc: 0.809 (best 0.809)
Epoch [13] | loss: 0.585, val acc: 0.816 (best 0.816)
Epoch [14] | loss: 0.755, val acc: 0.822 (best 0.822)
Epoch [15] | loss: 0.545, val acc: 0.829 (best 0.829)
Epo

## 鏈接預測 (Link Prediction)

在鏈接預測的任務中，我們輸入一張圖，並對圖上的每兩個點進行預測，所以我們不需要 `GraphDataLoader` 批次讀取圖資料，相反地，我們只要有圖的信息結構即可

Note: 我們以往的 sampler 都是在圖上選擇有關係的子圖，但是在鏈接預測的任務中，我們必須在訓練階段讓模型除了學習有鏈接的邊以外，也要學習無鏈接的邊

用數學的方式來講，如果一對點 $(u, v)$ 形成一條邊，那個這對點的得分應該要比以 $u$ 為 src，任選一個 $v'$ 作為 dst 的分數要來的高。從一個機率分配 $v'\sim P_v$ 中抽出 $v'$ 的過程稱為負採樣

- `dgl.dataloading.negative_sampler.PerSourceUniform(k)`
   - `k:` 每條邊的負樣本個數



In [ ]:
class PerSourceUniform(dgl.dataloading.negative_sampler._BaseNegativeSampler):
    """Negative sampler that randomly chooses negative destination nodes
    for each source node according to a uniform distribution.

    For each edge ``(u, v)`` of type ``(srctype, etype, dsttype)``, DGL generates
    :attr:`k` pairs of negative edges ``(u, v')``, where ``v'`` is chosen
    uniformly from all the nodes of type ``dsttype``.  The resulting edges will
    also have type ``(srctype, etype, dsttype)``.

    Parameters
    ----------
    k : int
        The number of negative samples per edge.

    Examples
    --------
    >>> g = dgl.graph(([0, 1, 2], [1, 2, 3]))
    >>> neg_sampler = dgl.dataloading.negative_sampler.PerSourceUniform(2)
    >>> neg_sampler(g, torch.tensor([0, 1]))
    (tensor([0, 0, 1, 1]), tensor([1, 0, 2, 3]))
    """

    def __init__(self, k):
        self.k = k

    def _generate(self, g, eids, canonical_etype):
        # dst 的型態 (dsttype)
        _, _, vtype = canonical_etype
        # 要抽多少個邊
        shape = F.shape(eids)
        # 邊的型態 (etype)
        dtype = F.dtype(eids)
        ctx = F.context(eids)
        shape = (shape[0] * self.k,)
        # find src
        src, _ = g.find_edges(eids, etype=canonical_etype)
        # sample exactly k negative sampler per src
        src = F.repeat(src, self.k, 0)
        # random select dst
        dst = F.randint(shape, dtype, ctx, 0, g.num_nodes(vtype))
        return src, dst

In [ ]:
g = dgl.graph(([0, 1, 2, 3, 4, 0, 1, 2, 3], [1, 2, 3, 4, 4, 4, 4, 4, 4]))
neg_sampler = dgl.dataloading.negative_sampler.PerSourceUniform(6)
neg_sampler(g, torch.tensor([0, 1]))

(tensor([0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1]),
 tensor([3, 4, 4, 0, 0, 1, 0, 0, 3, 3, 0, 2]))

### Custom Negative Sampler

若要自訂 `Sampler`，我們必須繼承於 `dgl.dataloading.negative_sampler._BaseNegativeSampler`，並且實現 `_generate` 方法，該方法會在 `_BaseNegativeSampler` 中被調用，或者自訂義類別，且需要實現 `__call__` 魔術方法，以下範例我們用每個點的 in_degrees 來影響抽樣機率

In [ ]:
class CustomSampler:

    def __init__(self, graph, k, alpha=0.75):
      self._g = graph
      self.weights = self._g.in_degrees().float() ** alpha
      self.k = k

    def __call__(self, graph, eids):
      src, _ = graph.find_edges(eids)
      # It's ok to use src.repeat(self.k)
      src = src.repeat_interleave(self.k)
      dst = self.weights.multinomial(len(src), replacement=True)
      return src, dst

In [ ]:
neg_sampler = CustomSampler(g, 6)
neg_sampler(g, torch.tensor([0, 4]))

(tensor([0, 0, 0, 0, 0, 0, 4, 4, 4, 4, 4, 4]),
 tensor([4, 4, 4, 1, 4, 4, 3, 1, 4, 4, 4, 4]))

In [ ]:
class CustomSampler(dgl.dataloading.negative_sampler._BaseNegativeSampler):

    def __init__(self, k, alpha=0.75):
      self.alpha = alpha
      self.k = k

    def _generate(self, graph, eids, canonical_etype):
      src, _ = graph.find_edges(eids)
      weights = graph.in_degrees().float() ** self.alpha
      src = src.repeat_interleave(self.k)
      dst = weights.multinomial(len(src), replacement=True)
      return src, dst

In [ ]:
neg_sampler = CustomSampler(6)
neg_sampler(g, torch.tensor([0, 4]))

(tensor([0, 0, 0, 0, 0, 0, 4, 4, 4, 4, 4, 4]),
 tensor([2, 4, 4, 3, 4, 4, 4, 4, 4, 4, 4, 1]))

### Negative Sampler DataLoader

由於我們的任務是對邊進行預測，所以我們必須先指定 `sampler` 在通過 `as_edge_prediction_sampler` 轉換成對邊抽樣的同時指定 `negative_sampler`

In [ ]:
neg_sampler = CustomSampler(6)
sampler = dgl.dataloading.MultiLayerFullNeighborSampler(2)
sampler = dgl.dataloading.as_edge_prediction_sampler(sampler, negative_sampler=neg_sampler)
dataloader = dgl.dataloading.DataLoader(
    g,
    [i for i in range(g.num_edges())],
    sampler,
    batch_size=4,
    shuffle=False,
    drop_last=False
)

[tensor([0, 1, 2, 3, 4]),
 Graph(num_nodes=5, num_edges=4,
       ndata_schemes={'_ID': Scheme(shape=(), dtype=torch.int64)}
       edata_schemes={'_ID': Scheme(shape=(), dtype=torch.int64)}),
 Graph(num_nodes=5, num_edges=24,
       ndata_schemes={'_ID': Scheme(shape=(), dtype=torch.int64)}
       edata_schemes={}),
 [Block(num_src_nodes=5, num_dst_nodes=5, num_edges=9),
  Block(num_src_nodes=5, num_dst_nodes=5, num_edges=9)]]


#### Unpacking

dataloader 是個 Iterator，回傳 `(input_nodes, pos_pair_graph, neg_pair_graph, blocks)` 的元組

- `input_nodes:` 輸入節點的 ID，數量同 `blocks[0].number_of_src_nodes()`
- `pos_pair_graph:` 抽樣邊的正樣本子圖 (edge subgraph) [num_edges=batch_size]
- `neg_pair_graph:` 抽樣邊的負樣本子圖 (edge subgraph) [num_edges=batch_size*k]
- `blocks:` 列表，`blocks[i]` 表示 GNN 中 $i$-layer 至 $(i+1)$-layer 的二分子圖

In [ ]:
next(iter(dataloader))

[tensor([0, 1, 2, 3, 4]),
 Graph(num_nodes=5, num_edges=4,
       ndata_schemes={'_ID': Scheme(shape=(), dtype=torch.int64)}
       edata_schemes={'_ID': Scheme(shape=(), dtype=torch.int64)}),
 Graph(num_nodes=5, num_edges=24,
       ndata_schemes={'_ID': Scheme(shape=(), dtype=torch.int64)}
       edata_schemes={}),
 [Block(num_src_nodes=5, num_dst_nodes=5, num_edges=9),
  Block(num_src_nodes=5, num_dst_nodes=5, num_edges=9)]]

### Example Dataset

我們鏈接預測一樣使用 `CoraGraphDataset`，這是一個非常著名的節點分類圖，該資料蒐集了 2708 個機器學習論文 (nodes)，每篇論文來自七個不同的機器學習主題 (神經網絡、概率方法、遺傳算法、監督學習、無監督學習、強化學習和規則學習)，節點代表論文之間的引用關係，更具體的說，有以下特徵

- 節點數量: 2708
- 邊的數量: 10556
- 節點特徵維度: 1433
- 類別數量: 7

In [ ]:
import dgl.function as fn

dataset = dgl.data.CoraGraphDataset()
g = dataset[0]

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.


In [ ]:
# utils.py
def _get_mask(n, ids):
  zeros = torch.zeros(n)
  zeros[ids] = 1
  return zeros

def _generate_edge_mask(graph, train_size=0.8, valid_size=0.1, test_size=0.1):
  src, dst = graph.edges()
  eids = np.arange(graph.num_edges())
  eids = np.random.permutation(eids)

  overall_size = train_size + valid_size + test_size
  train_size = train_size / overall_size
  valid_size = valid_size / overall_size
  test_size = test_size / overall_size

  train_size = int(len(eids) * train_size)
  valid_size = int(len(eids) * valid_size)
  test_size = int(len(eids) * test_size)

  train_eids = eids[:train_size]
  valid_eids = eids[train_size:(train_size+valid_size)]
  test_eids = eids[(train_size+valid_size):]

  graph.edata["train_edge_mask"] = _get_mask(graph.num_edges(), train_eids)
  graph.edata["valid_edge_mask"] = _get_mask(graph.num_edges(), valid_eids)
  graph.edata["test_edge_mask"] = _get_mask(graph.num_edges(), test_eids)

  return graph

  # train_src, train_dst = src[eids[:train_size]], dst[eids[:train_size]]
  # valid_src, valid_dst = src[eids[train_size:(train_size+valid_size)]], dst[eids[train_size:(train_size+valid_size)]]
  # test_src, test_dst = src[eids[(train_size+valid_size):(train_size+valid_size+test_size)]], dst[eids[(train_size+valid_size):(train_size+valid_size+test_size)]]

  # train_g = dgl.graph((train_src, train_dst), num_nodes=graph.num_nodes())
  # valid_g = dgl.graph((valid_src, valid_dst), num_nodes=graph.num_nodes())
  # test_g = dgl.graph((test_src, test_dst), num_nodes=graph.num_nodes())

  # return train_g, valid_g, test_g

In [ ]:
class StochasticTwoLayerGCN(nn.Module):
    def __init__(self, in_dim, hid_dim, out_dim):
        """两层的GCN模型"""
        super().__init__()
        self.conv1 = dglnn.GraphConv(in_dim, hid_dim, allow_zero_in_degree=True)
        self.conv2 = dglnn.GraphConv(hid_dim, out_dim, allow_zero_in_degree=True)

    def forward(self, blocks, x):
        x = F.relu(self.conv1(blocks[0], x))
        x = F.relu(self.conv2(blocks[1], x))
        return x

class DotPredictor(nn.Module):
    def forward(self, g, h):
        with g.local_scope():
            g.ndata["h"] = h
            # Compute a new edge feature named 'score' by a dot-product between the
            # source node feature 'h' and destination node feature 'h'.
            g.apply_edges(fn.u_dot_v("h", "h", "score"))
            # u_dot_v returns a 1-element vector for each edge so you need to squeeze it.
            return g.edata["score"][:, 0]

class MLPPredictor(nn.Module):
    def __init__(self, h_feats):
        super().__init__()
        self.W1 = nn.Linear(h_feats * 2, h_feats)
        self.W2 = nn.Linear(h_feats, 1)

    def apply_edges(self, edges):
        """
        Computes a scalar score for each edge of the given graph.

        Parameters
        ----------
        edges :
            Has three members ``src``, ``dst`` and ``data``, each of
            which is a dictionary representing the features of the
            source nodes, the destination nodes, and the edges
            themselves.

        Returns
        -------
        dict
            A dictionary of new edge features.
        """
        h = torch.cat([edges.src["h"], edges.dst["h"]], 1)
        return {"score": self.W2(F.relu(self.W1(h))).squeeze(1)}

    def forward(self, g, h):
        with g.local_scope():
            g.ndata["h"] = h
            g.apply_edges(self.apply_edges)
            return g.edata["score"]

class Model(nn.Module):
    def __init__(self, in_features, hidden_features, out_features):
      super().__init__()
      self.gcn = StochasticTwoLayerGCN(in_features, hidden_features, out_features)
      self.predictor = DotPredictor()

    def forward(self, input_nodes, pos_pair_graph, neg_pair_graph, blocks):
        input_nodes = self.gcn(blocks, input_nodes)
        pos_score = self.predictor(pos_pair_graph, input_nodes)
        neg_score = self.predictor(neg_pair_graph, input_nodes)
        return pos_score, neg_score

def compute_loss(pos_score, neg_score):
    n = pos_score.shape[0]
    return (neg_score.view(n, -1) - pos_score.view(n, -1) + 1).clamp(min=0).mean()

def compute_auc(pos_score, neg_score):
    pos_score = torch.tensor(pos_score)
    neg_score = torch.tensor(neg_score)
    scores = torch.cat([pos_score, neg_score]).numpy()
    labels = torch.cat(
        [torch.ones(pos_score.shape[0]), torch.zeros(neg_score.shape[0])]
    ).numpy()
    return roc_auc_score(labels, scores)

In [ ]:
# main.py
import argparse
import sys
from sklearn.metrics import roc_auc_score
import warnings

warnings.filterwarnings("ignore")

sys.argv = ['main.py']

def parse_args():
    parser = argparse.ArgumentParser(description='DGL GNN Heterograph node classification')
    parser.add_argument('--epochs', type=int, default=50, help='number of training epochs')
    parser.add_argument('--batch_size', type=int, default=512, help='batch size')
    parser.add_argument('--num_negatives', type=int, default=6, help='number of negative per edge')
    parser.add_argument("--print_freq", type=int, default=1, help="print frequency")
    parser.add_argument('--save_name', type=str, default='model.pth', help='name of the file to save the trained model')
    parser.add_argument('--lr', type=float, default=1e-2, help='learning rate for the optimizer')
    parser.add_argument('--weight_decay', type=float, default=0, help='weight decay for the optimizer')
    parser.add_argument('--resume', type=bool, default=False, help='resume or not')
    parser.add_argument('--resume_path', type=str, default="best_model.pth", help='resume path')
    parser.add_argument('--num_classes', type=int, default=7, help='number of classes')
    parser.add_argument('--hidden_feats', type=int, default=16, help='number of hidden units')
    parser.add_argument('--model_name', type=str, default="tv_densenet121", help='model name')
    parser.add_argument('--device', type=str, default="cuda", help='device')
    parser.add_argument('--rank', type=int, default=0, help='the rank of the device, 0 if single GPU')
    parser.add_argument('--name', type=str, default="test", help='name for the wandb run')
    parser.add_argument('--fanout', type=list, default=[10,25], help='number of samples in training steps')
    args = parser.parse_args()
    return args

def main():

    seed_everything(42)

    args = parse_args()

    # d = vars(args)
    # key = "Your API key"
    # wandb_settings(key, d, "DGL-Cora-Heterograph-Node-Classification", "DDCVLAB", args.name)

    dataset = dgl.data.CoraGraphDataset()
    graph = dataset[0]
    graph = _generate_edge_mask(graph)

    train_mask = graph.edata['train_edge_mask']
    val_mask = graph.edata['valid_edge_mask']
    test_mask = graph.edata['test_edge_mask']

    sampler = dgl.dataloading.NeighborSampler(args.fanout)
    neg_sampler = dgl.dataloading.negative_sampler.PerSourceUniform(args.num_negatives)
    sampler = dgl.dataloading.as_edge_prediction_sampler(sampler, negative_sampler=neg_sampler)
    train_dataloader = dgl.dataloading.DataLoader(
        graph,
        get_id(train_mask),
        sampler,
        batch_size=args.batch_size,
        shuffle=True,
        drop_last=False,
    )
    sampler = dgl.dataloading.MultiLayerFullNeighborSampler(2)
    neg_sampler = dgl.dataloading.negative_sampler.PerSourceUniform(args.num_negatives)
    sampler = dgl.dataloading.as_edge_prediction_sampler(sampler, negative_sampler=neg_sampler)
    valid_dataloader = dgl.dataloading.DataLoader(
        graph,
        get_id(val_mask),
        sampler,
        batch_size = args.batch_size
    )
    sampler = dgl.dataloading.MultiLayerFullNeighborSampler(2)
    neg_sampler = dgl.dataloading.negative_sampler.PerSourceUniform(args.num_negatives)
    sampler = dgl.dataloading.as_edge_prediction_sampler(sampler, negative_sampler=neg_sampler)
    test_dataloader = dgl.dataloading.DataLoader(
        graph,
        get_id(test_mask),
        sampler,
        batch_size = args.batch_size
    )


    model = Model(graph.ndata["feat"].shape[1], args.hidden_feats, args.hidden_feats)
    if args.resume:
      state_dict = torch.load(args.resume_path)
      model.load_state_dict(state_dict, strict=True)

    device = args.device
    model.to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=args.lr, weight_decay=args.weight_decay)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.1, patience=5)

    best_val_auc = 0
    pos_scores, neg_scores = [], []
    train_epoch_loss = 0
    valid_epoch_loss = 0
    for epoch in range(args.epochs):
        model.train()
        for step, (input_nodes, pos_pair_graph, neg_pair_graph, blocks) in enumerate(train_dataloader):
            # Forward
            pos_pair_graph = pos_pair_graph.to(device)
            neg_pair_graph = neg_pair_graph.to(device)
            blocks = [block.to(device) for block in blocks]
            input_features = blocks[0].srcdata["feat"]
            # input_nodes = input_nodes.to(device)

            pos_score, neg_score = model(input_features, pos_pair_graph, neg_pair_graph, blocks)
            train_loss = compute_loss(pos_score, neg_score)
            train_epoch_loss += train_loss.item()

            # Compute accuracy on training/validation/test
            pos_scores.extend(pos_score)
            neg_scores.extend(neg_score)

            # Backward
            optimizer.zero_grad()
            train_loss.backward()
            optimizer.step()

        train_auc = compute_auc(pos_scores, neg_scores)

        model.eval()
        with torch.no_grad():
            pos_scores, neg_scores = [], []
            for step, (input_nodes, pos_pair_graph, neg_pair_graph, blocks) in enumerate(valid_dataloader):
                pos_pair_graph = pos_pair_graph.to(device)
                neg_pair_graph = neg_pair_graph.to(device)
                blocks = [block.to(device) for block in blocks]
                # input_nodes = input_nodes.to(device)
                input_features = blocks[0].srcdata["feat"]
                pos_score, neg_score = model(input_features, pos_pair_graph, neg_pair_graph, blocks)
                valid_loss = compute_loss(pos_score, neg_score)
                valid_epoch_loss += valid_loss.item()
                pos_scores.extend(pos_score)
                neg_scores.extend(neg_score)

        val_auc = compute_auc(pos_scores, neg_scores)

        # Save the best validation accuracy and the corresponding test accuracy.
        if best_val_auc < val_auc:
            best_val_auc = val_auc
            torch.save(model.state_dict(), args.save_name)
        if epoch % args.print_freq == 0:
            print(
                "Epoch [{}] | loss: {:.3f}, val acc: {:.3f} (best {:.3f})".format(
                    epoch, valid_loss, val_auc, best_val_auc
                )
            )
        # wandb.log({"Train ACC": train_acc, "Epoch": epoch})
        # wandb.log({"Val ACC": val_acc, "Epoch": epoch})
        # wandb.log({"Train Loss": train_loss, "Epoch": epoch})
        # wandb.log({"Val Loss": valid_loss, "Epoch": epoch})

        scheduler.step(valid_loss)

    # Testing
    state_dict = torch.load(args.save_name)
    model.load_state_dict(state_dict)
    model.eval()

    trn_label, trn_pred = [], []
    with torch.no_grad():
        for step, (input_nodes, pos_pair_graph, neg_pair_graph, blocks) in enumerate(test_dataloader):
            pos_pair_graph = pos_pair_graph.to(device)
            neg_pair_graph = neg_pair_graph.to(device)
            blocks = [block.to(device) for block in blocks]
            input_features = blocks[0].srcdata["feat"]
            # input_nodes = input_nodes.to(device)
            pos_score, neg_score = model(input_features, pos_pair_graph, neg_pair_graph, blocks)
            pos_scores.extend(pos_score)
            neg_scores.extend(neg_score)

    test_acc = compute_auc(pos_scores, neg_scores)

    print(f"Test ACC: {test_acc}")
    # wandb.log({"Test ACC": test_acc})

if __name__ == "__main__":
    main()

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.
Epoch [0] | loss: 1.472, val acc: 0.637 (best 0.637)
Epoch [1] | loss: 1.625, val acc: 0.634 (best 0.637)
Epoch [2] | loss: 1.594, val acc: 0.647 (best 0.647)
Epoch [3] | loss: 0.786, val acc: 0.768 (best 0.768)
Epoch [4] | loss: 0.630, val acc: 0.837 (best 0.837)
Epoch [5] | loss: 0.592, val acc: 0.853 (best 0.853)
Epoch [6] | loss: 0.452, val acc: 0.857 (best 0.857)
Epoch [7] | loss: 0.509, val acc: 0.869 (best 0.869)
Epoch [8] | loss: 0.492, val acc: 0.881 (best 0.881)
Epoch [9] | loss: 0.514, val acc: 0.891 (best 0.891)
Epoch [10] | loss: 0.233, val acc: 0.905 (best 0.905)
Epoch [11] | loss: 0.414, val acc: 0.905 (best 0.905)
Epoch [12] | loss: 0.432, val acc: 0.909 (best 0.909)
Epoch [13] | loss: 0.291, val acc: 0.913 (best 0.913)
Epoch [14] | loss: 0.525, val acc: 0.912 (best 0.913)
Epoch [15] | loss:

## 整圖分類 (Graph Classification)

在整圖分類的任務中，我們批次輸入圖，再輸入過程中，我們使用 `GraphDataLoader` 批次讀取圖資料，並用 `torch.utils.data.sampler` 對批次的圖進行抽樣

`Sampler` 是一個帶有 `__len__` 的 iterator，如果需要自定義 Sampler 需要包含三個部分

1. `__init__:` 初始化設定
2. `__iter__:` 每次 iteration 中抽取 indices 的方法，回傳一個包含 indices 的 list，通常用 yield 方法
3. `__len__:` 數據長度


### Example Dataset

我們使用 `FakeNewsDataset`，該數據集來自於 Twitter 用戶的假新聞預測，介紹如下:

該數據集由從 Twitter 提取的兩組樹狀結構的真實/虛假新聞傳播圖組成。與大多數用於圖分類任務的基準數據集不同，該數據集中的圖是有向樹狀結構圖，其中根節點代表新聞，葉節點是轉發根新聞的 Twitter 用戶。此外，節點特徵是使用不同的預訓練語言模型對用戶歷史推文進行編碼：

- BERT: 768 維度
- content: 300 維度的 Spacy + 10 維度的 profile
- Spacy: 300 維度
- profile: 10 維度

In [ ]:
dataset = dgl.data.FakeNewsDataset('politifact', 'bert')
graph, label = dataset[1]
num_classes = dataset.num_classes
feat = dataset.feature
labels = dataset.labels
print(dataset)

Dataset("politifact", num_graphs=314, save_path=/root/.dgl/politifact_505d0a2f)


In [ ]:
graph.ndata

{'_ID': tensor([497, 498, 499, 500, 501, 502, 503, 504, 505, 506, 507, 508, 509, 510,
        511, 512, 513, 514, 515, 516, 517])}

In [ ]:
from dgl.dataloading import GraphDataLoader

In [ ]:
from torch.utils.data.sampler import SubsetRandomSampler

def _generate_dataloader(dataset, batch_size=16, train_size=.8, valid_size=.1, test_size=.1):

    gids = np.arange(len(dataset))
    gids = np.random.permutation(gids)

    overall_size = train_size + valid_size + test_size
    train_size = train_size / overall_size
    valid_size = valid_size / overall_size
    test_size = test_size / overall_size

    train_size = int(len(gids) * train_size)
    valid_size = int(len(gids) * valid_size)
    test_size = int(len(gids) * test_size)

    train_sampler = SubsetRandomSampler(torch.arange(train_size))
    valid_sampler = SubsetRandomSampler(torch.arange(train_size, train_size+valid_size))
    test_sampler = SubsetRandomSampler(torch.arange(train_size+valid_size, len(gids)))

    train_dataloader = GraphDataLoader(
        dataset, sampler=train_sampler, batch_size=batch_size, drop_last=False
    )
    valid_dataloader = GraphDataLoader(
        dataset, sampler=valid_sampler, batch_size=batch_size, drop_last=False
    )
    test_dataloader = GraphDataLoader(
        dataset, sampler=test_sampler, batch_size=batch_size, drop_last=False
    )

    return train_dataloader, valid_dataloader, test_dataloader

In [ ]:
from dgl.nn import GraphConv


class GCN(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GraphConv(in_feats, h_feats, allow_zero_in_degree=True)
        self.conv2 = GraphConv(h_feats, num_classes, allow_zero_in_degree=True)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        g.ndata["h"] = h
        return dgl.mean_nodes(g, "h")

In [ ]:
# Create the model with given dimensions
model = GCN(768, 100, 2)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

for epoch in range(20):
    for batched_graph, labels in train_dataloader:
        pred = model(batched_graph, feat[batched_graph.ndata["_ID"]].float())
        loss = F.cross_entropy(pred, labels.long())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

num_correct = 0
num_tests = 0
for batched_graph, labels in test_dataloader:
    pred = model(batched_graph, feat[batched_graph.ndata["_ID"]].float().float())
    num_correct += (pred.argmax(1) == labels).sum().item()
    num_tests += len(labels)

print("Test accuracy:", num_correct / num_tests)

Test accuracy: 0.7936507936507936


In [ ]:
# main.py
import argparse
import sys
from sklearn.metrics import accuracy_score
import warnings

warnings.filterwarnings("ignore")

sys.argv = ['main.py']

def parse_args():
    parser = argparse.ArgumentParser(description='DGL GNN Heterograph node classification')
    parser.add_argument('--epochs', type=int, default=50, help='number of training epochs')
    parser.add_argument('--batch_size', type=int, default=32, help='batch size')
    parser.add_argument("--print_freq", type=int, default=1, help="print frequency")
    parser.add_argument('--save_name', type=str, default='model.pth', help='name of the file to save the trained model')
    parser.add_argument('--lr', type=float, default=1e-2, help='learning rate for the optimizer')
    parser.add_argument('--weight_decay', type=float, default=0, help='weight decay for the optimizer')
    parser.add_argument('--resume', type=bool, default=False, help='resume or not')
    parser.add_argument('--resume_path', type=str, default="best_model.pth", help='resume path')
    parser.add_argument('--num_classes', type=int, default=7, help='number of classes')
    parser.add_argument('--hidden_feats', type=int, default=16, help='number of hidden units')
    parser.add_argument('--model_name', type=str, default="tv_densenet121", help='model name')
    parser.add_argument('--device', type=str, default="cuda", help='device')
    parser.add_argument('--rank', type=int, default=0, help='the rank of the device, 0 if single GPU')
    parser.add_argument('--name', type=str, default="test", help='name for the wandb run')
    parser.add_argument('--fanout', type=list, default=[10,25], help='number of samples in training steps')
    args = parser.parse_args()
    return args

def main():

    seed_everything(42)

    args = parse_args()

    # d = vars(args)
    # key = "Your API key"
    # wandb_settings(key, d, "DGL-Cora-Heterograph-Node-Classification", "DDCVLAB", args.name)

    dataset = dgl.data.FakeNewsDataset('politifact', 'bert')
    num_classes = dataset.num_classes
    feat = dataset.feature

    train_dataloader, valid_dataloader, test_dataloader = _generate_dataloader(dataset, args.batch_size)

    model = GCN(feat.shape[1], args.hidden_feats, num_classes)
    if args.resume:
      state_dict = torch.load(args.resume_path)
      model.load_state_dict(state_dict, strict=True)

    device = args.device
    model.to(device)
    feat = feat.to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=args.lr, weight_decay=args.weight_decay)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.1, patience=5)

    best_val_auc = 0
    trn_label, trn_pred = [], []
    train_epoch_loss = 0
    valid_epoch_loss = 0
    for epoch in range(args.epochs):
        model.train()
        for step, (batched_graph, labels) in enumerate(train_dataloader):
            # Forward
            batched_graph = batched_graph.to(device)
            labels = labels.to(device)
            input_features = feat[batched_graph.ndata["_ID"]].float()

            logits = model(batched_graph, input_features)
            train_loss = F.cross_entropy(logits, labels.long())
            train_epoch_loss += train_loss.item()

            # Compute accuracy on training/validation/test
            trn_label.extend(labels.cpu().numpy().tolist())
            trn_pred.extend(logits.argmax(1).cpu().numpy().tolist())

            # Backward
            optimizer.zero_grad()
            train_loss.backward()
            optimizer.step()

        train_auc = accuracy_score(trn_label, trn_pred)

        model.eval()
        with torch.no_grad():
            trn_label, trn_pred = [], []
            for step, (batched_graph, labels) in enumerate(valid_dataloader):

                batched_graph = batched_graph.to(device)
                labels = labels.to(device)
                input_features = feat[batched_graph.ndata["_ID"]].float()

                logits = model(batched_graph, input_features)
                valid_loss = F.cross_entropy(logits, labels.long())
                valid_epoch_loss += valid_loss.item()
                trn_label.extend(labels.cpu().numpy().tolist())
                trn_pred.extend(logits.argmax(1).cpu().numpy().tolist())

        val_auc = accuracy_score(trn_label, trn_pred)

        # Save the best validation accuracy and the corresponding test accuracy.
        if best_val_auc < val_auc:
            best_val_auc = val_auc
            torch.save(model.state_dict(), args.save_name)
        if epoch % args.print_freq == 0:
            print(
                "Epoch [{}] | loss: {:.3f}, val acc: {:.3f} (best {:.3f})".format(
                    epoch, valid_loss, val_auc, best_val_auc
                )
            )
        # wandb.log({"Train ACC": train_acc, "Epoch": epoch})
        # wandb.log({"Val ACC": val_acc, "Epoch": epoch})
        # wandb.log({"Train Loss": train_loss, "Epoch": epoch})
        # wandb.log({"Val Loss": valid_loss, "Epoch": epoch})

        scheduler.step(valid_loss)

    # Testing
    state_dict = torch.load(args.save_name)
    model.load_state_dict(state_dict)
    model.eval()

    trn_label, trn_pred = [], []
    with torch.no_grad():
        for step, (batched_graph, labels) in enumerate(test_dataloader):
            batched_graph = batched_graph.to(device)
            labels = labels.to(device)
            input_features = feat[batched_graph.ndata["_ID"]].float()

            logits = model(batched_graph, input_features)
            trn_label.extend(labels.cpu().numpy().tolist())
            trn_pred.extend(logits.argmax(1).cpu().numpy().tolist())

    test_acc = accuracy_score(trn_label, trn_pred)

    print(f"Test ACC: {test_acc}")
    # wandb.log({"Test ACC": test_acc})

if __name__ == "__main__":
    main()

Epoch [0] | loss: 0.904, val acc: 0.000 (best 0.000)
Epoch [1] | loss: 0.876, val acc: 0.000 (best 0.000)
Epoch [2] | loss: 0.909, val acc: 0.000 (best 0.000)
Epoch [3] | loss: 0.924, val acc: 0.000 (best 0.000)
Epoch [4] | loss: 0.815, val acc: 0.032 (best 0.032)
Epoch [5] | loss: 0.822, val acc: 0.065 (best 0.065)
Epoch [6] | loss: 0.860, val acc: 0.065 (best 0.065)
Epoch [7] | loss: 0.773, val acc: 0.290 (best 0.290)
Epoch [8] | loss: 0.767, val acc: 0.355 (best 0.355)
Epoch [9] | loss: 0.849, val acc: 0.290 (best 0.355)
Epoch [10] | loss: 0.538, val acc: 0.774 (best 0.774)
Epoch [11] | loss: 1.039, val acc: 0.129 (best 0.774)
Epoch [12] | loss: 0.392, val acc: 0.968 (best 0.968)
Epoch [13] | loss: 0.944, val acc: 0.258 (best 0.968)
Epoch [14] | loss: 0.351, val acc: 0.968 (best 0.968)
Epoch [15] | loss: 0.814, val acc: 0.419 (best 0.968)
Epoch [16] | loss: 0.423, val acc: 0.839 (best 0.968)
Epoch [17] | loss: 0.743, val acc: 0.484 (best 0.968)
Epoch [18] | loss: 0.410, val acc: 0.8